<a href="https://colab.research.google.com/github/carneiroDotDev/agentic-ai-ws/blob/main/ADK_Learning_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Welcome to Your ADK Adventure - Tools & Memory! 🚀

Welcome, Agent Architect! This notebook is your guide to giving your AI agents two essential superpowers: custom tools and conversational memory.

By the end of this adventure, you will be able to:

- **Build a Foundational Agent**: Create a simple but effective AI agent from scratch using the Google Agent Development Kit (ADK).

- **Grant New Skills with Custom Tools**: Teach an agent to perform new tasks by connecting it to external APIs, like a real-time weather service.

- **Create a Team of Agents**: Assemble a multi-agent system where a primary agent can delegate specialized tasks to other agents.

- **Master Conversational Memory**: Understand the critical role of Sessions in enabling agents to remember previous interactions, handle feedback, and carry on a coherent conversation.


Let's get this adventure started!

## Author

HI, I'm Qingyue (Annie) Wang, a developer advocate and AI engineer at **Google**, passionate about helping developers build with AI and cloud technologies :)


If you have questions with this notebook, contact me on [LinkedIn](https://www.linkedin.com/in/qingyuewang/) , [X](https://twitter.com/qingyuewang) or email anniewangtech0510@Gmail.com


```
  (\__/)
  (•ㅅ•)
  /づ  📚      Enjoy learning AI Agents :)
```


-------------
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

👉 **Get Your API Key HERE**: https://codelabs.developers.google.com/onramp/instructions#1

 -----------------------------------------------------------------------------

```
 ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️
   /\_/\     /\_/\     /\_/\      /\_/\       /\_/\
  ( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )    ( o_o )             
```


## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries and securely configures your Google API key so your agents can access the power of Gemini.

In [2]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

/Users/luizcarneiro/Documents/Coding/projects/agentic-ai-ws/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries are ready to go!


In [3]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

✅ API Key configured successfully! Let the fun begin.


---
## Part 1: Your First Agent - The Day Trip Genie 🧞

Meet your first creation! The `day_trip_agent` is a simple but powerful assistant. We're making it a little smarter by teaching it to understand **budget constraints**.

* **Agent**: The brain of the operation, defined by its instructions, tools, and the AI model it uses.
* **Session**: The conversation history. For this simple agent, it's just a container for a single request-response.
* **Runner**: The engine that connects the `Agent` and the `Session` to process your request and get a response.

```
+--------------------------------------------------+
|         Spontaneous Day Trip Agent 🤖            |
|--------------------------------------------------|
|  Model: gemini-2.5-flash                         |
|  Description:                                    |
|   Generates full-day trip itineraries based on   |
|   mood, interests, and budget                    |
|--------------------------------------------------|
|  🔧 Tools:                                       |
|   - Google Search                                |
|--------------------------------------------------|
|  🧠 Capabilities:                                |
|   - Budget Awareness (cheap / splurge)           |
|   - Mood Matching (adventurous, relaxing, etc.)  |
|   - Real-Time Info (hours, events)               |
|   - Morning / Afternoon / Evening plan           |
+--------------------------------------------------+

            ▲
            |
    +------------------+
    |   User Input     |
    |------------------|
    |  Mood            |
    |  Interests       |
    |  Budget          |
    +------------------+

            |
            ▼

+--------------------------------------------------+
|             Output: Markdown Itinerary           |
|--------------------------------------------------|
| - Time blocks (Morning / Afternoon / Evening)    |
| - Venue names with links and hours               |
| - Budget-matching activities                     |
+--------------------------------------------------+
```


In [4]:
# --- Agent Definition ---

def create_day_trip_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
        instruction="""
        You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

        Your Mission:
        Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

        Guidelines:
        1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
        2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
        3. **Real-Time Focus**: Search for current operating hours and special events.
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
        """,
        tools=[google_search]
    )

day_trip_agent = create_day_trip_agent()
print(f"🧞 Agent '{day_trip_agent.name}' is created and ready for adventure!")

🧞 Agent 'day_trip_agent' is created and ready for adventure!


In [5]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [6]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=day_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a relaxing and artsy day trip near Sunnyvale, CA. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(day_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

🗣️ User Query: 'Plan a relaxing and artsy day trip near Sunnyvale, CA. Keep it affordable!'

🚀 Running query for agent: 'day_trip_agent' in session: '36f921e6-5941-4fcd-bc26-2bd6fdccc2a4'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Here's a relaxing and artsy day trip itinerary near Sunnyvale, CA, designed to be affordable and enjoyable for a Friday, November 21, 2025:

---

## Relaxing & Artsy Day Trip: Sunnyvale & Nearby Gems

Embrace a day of artistic exploration and tranquil moments without breaking the bank! This itinerary focuses on free or low-cost activities, perfect for unwinding and soaking in local culture.

---

### **Morning (10:30 AM - 1:00 PM): Artistic Immersion**

*   **Triton Museum of Art (Santa Clara)**
    *   Start your day with a visit to the Triton Museum of Art, which offers **free admission** and **free parking** to its exhibitions. This allows for a budget-friendly art experience. The museum features a diverse

Here's a relaxing and artsy day trip itinerary near Sunnyvale, CA, designed to be affordable and enjoyable for a Friday, November 21, 2025:

---

## Relaxing & Artsy Day Trip: Sunnyvale & Nearby Gems

Embrace a day of artistic exploration and tranquil moments without breaking the bank! This itinerary focuses on free or low-cost activities, perfect for unwinding and soaking in local culture.

---

### **Morning (10:30 AM - 1:00 PM): Artistic Immersion**

*   **Triton Museum of Art (Santa Clara)**
    *   Start your day with a visit to the Triton Museum of Art, which offers **free admission** and **free parking** to its exhibitions. This allows for a budget-friendly art experience. The museum features a diverse range of art, often focusing on California artists. Take your time to wander through the galleries and appreciate the various styles and mediums on display. Exhibitions like "Urban Visions: Life in Motion by Dean Larson" and "Shifting Terrain by Mark Engel" are currently on view.
    *   *Location:* 1505 Warburton Ave, Santa Clara, CA 95050
    *   *Cost:* Free admission, free parking.
    *   *Operating Hours:* Open Tuesday - Sunday, 11 AM - 5 PM (verify on their website for any special Friday changes).

### **Lunch (1:00 PM - 2:30 PM): Serene Picnic & Nature's Art**

*   **Japanese Friendship Garden (San Jose)**
    *   Pack a delicious, affordable picnic lunch to enjoy at the serene Japanese Friendship Garden, located in Kelley Park in San Jose. This beautiful 6-acre garden is a living work of art, featuring meticulously manicured landscapes, tranquil ponds, and charming bridges. It's an ideal spot to relax, reflect, and enjoy your meal amidst natural beauty. Parking is also free.
    *   *Location:* 1300 Senter Rd, San Jose, CA 95112 (part of Kelley Park)
    *   *Cost:* Free admission, free parking.
    *   *Operating Hours:* Open daily from dawn to dusk.

### **Afternoon (2:30 PM - 5:00 PM): Local Public Art Stroll**

*   **Sunnyvale Public Art Self-Guided Tour**
    *   Return to Sunnyvale for a relaxed self-guided public art tour. Sunnyvale boasts over 200 public art pieces throughout the city, including an interactive online map and downloadable walking tour maps for areas like Civic Center, Downtown, and various neighborhoods. You can explore sculptures, murals, and installations, including the "Sun Flair" program which features 26 identical sun sculptures transformed by local artists, displayed in city parks. This offers a lovely, free way to enjoy art outdoors and discover hidden gems in the city.
    *   *Location:* Various locations in Sunnyvale. Start with a map from Sunnyvale.ca.gov/PublicArt.
    *   *Cost:* Free.
    *   *Operating Hours:* Accessible from sunrise to sunset in public areas.

### **Evening (5:00 PM - 8:00 PM): Sunset Views & Cozy Comfort**

*   **Palo Alto Baylands Nature Preserve - Sunset Walk**
    *   Head to the Palo Alto Baylands Nature Preserve for a relaxing walk as the day winds down and to catch a beautiful sunset. This preserve offers flat trails winding through wetlands and salt marshes, providing a "one-of-a-kind sunset view" where you can see the full sky glow. It’s a peaceful way to conclude your day of art and relaxation.
    *   *Location:* 2775 Embarcadero Rd, Palo Alto, CA 94303
    *   *Cost:* Free.
    *   *Operating Hours:* Open from 8 AM until 30 minutes after sunset.

*   **Cozy Coffee & Light Bites in Downtown Sunnyvale**
    *   After the sunset, make your way to downtown Sunnyvale for a warm drink and a light, affordable dinner or snack.
        *   **Bean Scene Cafe** is known as a place where people "meet, laugh, and relax," offering a comfortable atmosphere.
        *   Alternatively, **Coffee & More** in Downtown Sunnyvale has a "cozy atmosphere" and is a local favorite.
        *   **Philz Coffee** is another option with "comfy couches and chairs," ideal for unwinding.
    *   Enjoy a warm beverage and reflect on your relaxing and artsy day.
    *   *Location:* Various cafes in Downtown Sunnyvale (e.g., Murphy Ave).
    *   *Cost:* Affordable, depending on your choices.

---

Enjoy your spontaneous, affordable, relaxing, and artsy day trip!

--------------------------------------------------



---
## Part 2: Supercharging Agents with Custom Tools 🛠️

So far, we've used the powerful built-in `GoogleSearch` tool. But the true power of agents comes from connecting them to your own logic and data sources.

This is where **custom tools** come in. Let's explore three patterns for giving your agent new skills, using real-world, practical examples.

### 2.1 The Simple `FunctionTool`: Calling a Real-Time Weather API

The most direct way to create a tool is by writing a Python function. This is perfect for synchronous tasks like fetching data from an API.

**Key Concept:** The function's **docstring** is critical. The ADK uses it as the tool's official description, which the LLM reads to understand its purpose, parameters, and when to use it.

In this example, we'll create a tool that calls the **free, public U.S. National Weather Service API** to get a real-time forecast. No API key needed!

In [7]:
# --- Tool Definition: A function that calls a live public API ---
import requests
import json

# A simple lookup to avoid needing a separate geocoding API for this example
LOCATION_COORDINATES = {
    "sunnyvale": "37.3688,-122.0363",
    "san francisco": "37.7749,-122.4194",
    "lake tahoe": "39.0968,-120.0324"
}

def get_live_weather_forecast(location: str) -> dict:
    """Gets the current, real-time weather forecast for a specified location in the US.

    Args:
        location: The city name, e.g., "San Francisco".

    Returns:
        A dictionary containing the temperature and a detailed forecast.
    """
    print(f"🛠️ TOOL CALLED: get_live_weather_forecast(location='{location}')")

    # Find coordinates for the location
    normalized_location = location.lower()
    coords_str = None
    for key, val in LOCATION_COORDINATES.items():
        if key in normalized_location:
            coords_str = val
            break
    if not coords_str:
        return {"status": "error", "message": f"I don't have coordinates for {location}."}

    try:
        # NWS API requires 2 steps: 1. Get the forecast URL from the coordinates.
        points_url = f"https://api.weather.gov/points/{coords_str}"
        headers = {"User-Agent": "ADK Example Notebook"}
        points_response = requests.get(points_url, headers=headers)
        points_response.raise_for_status() # Raise an exception for bad status codes
        forecast_url = points_response.json()['properties']['forecast']

        # 2. Get the actual forecast from the URL.
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()

        # Extract the relevant forecast details
        current_period = forecast_response.json()['properties']['periods'][0]
        return {
            "status": "success",
            "temperature": f"{current_period['temperature']}°{current_period['temperatureUnit']}",
            "forecast": current_period['detailedForecast']
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Agent Definition: An agent that USES the new tool ---

weather_agent = Agent(
    name="weather_aware_planner",
    model="gemini-2.5-flash",
    description="A trip planner that checks the real-time weather before making suggestions.",
    instruction="You are a cautious trip planner. Before suggesting any outdoor activities, you MUST use the `get_live_weather_forecast` tool to check conditions. Incorporate the live weather details into your recommendation.",
    tools=[get_live_weather_forecast]
)

print(f"🌦️ Agent '{weather_agent.name}' is created and can now call a live weather API!")

🌦️ Agent 'weather_aware_planner' is created and can now call a live weather API!


In [8]:
# --- Let's test the Weather-Aware Planner ---

async def run_weather_planner_test():
    weather_session = await session_service.create_session(app_name=weather_agent.name, user_id=my_user_id)
    query = "I want to go hiking near Lake Tahoe, what's the weather like?"
    print(f"🗣️ User Query: '{query}'")
    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_planner_test()

🗣️ User Query: 'I want to go hiking near Lake Tahoe, what's the weather like?'

🚀 Running query for agent: 'weather_aware_planner' in session: '8152db20-fdc6-4e0b-a61a-1d7e6c53790d'...


EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'location': 'Lake Tahoe'
        },
        id='adk-6cfe188b-aae2-4e40-805e-b8cd11200766',
        name='get_live_weather_forecast'
      ),
      thought_signature=b'\n\xbe\x02\x01\xd1\xed\x8ao\xdd\xe6\x06\x87\xbf\xe67}LI\xd6\x18\x8e\x16\xe8\xc6f\x9a5\xa24JG:h\xa4\xca\xb5\x86\xf3\xd4Lq\xe4\xb8\xc7s\x0b\x94\xf3\xd0oSO\xa4\xb1\x18qL\x87\xdfj\xbc/Y\x12\x12\xb7\x95\xd1\x06V\x99\x88\xcb^\xd2;B=\x03S\xfd\xaf\r\xf7\xbd}\x87\xb8\x1b\x1aZ\x16YYj\xc0\x00...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=20,
  prompt_token_count=185,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      t

It's currently 39°F and sunny with an east wind around 10 mph near Lake Tahoe. It sounds like a beautiful day for a hike! Enjoy the outdoors!

--------------------------------------------------



## 2.2 The Agent-as-a-Tool: Consulting a Specialist 🧑‍🍳

Why build one agent that does everything when you can build a **team of specialist agents?** The **Agent-as-a-Tool** pattern allows one agent to delegate a task to another agent.

**Key Concept:** This is different from a sub-agent. When Agent A calls Agent B as a tool, Agent B's response is passed **back to Agent A**. Agent A then uses that information to form its own final response to the user. It's a powerful way to compose complex behaviors from simpler, focused, and reusable agents.

### How It Works

Our top-level agent, the `trip_data_concierge_agent`, acts as the **Orchestrator**. It has two tools at its disposal:

1.  `call_db_agent`: A function that internally calls our `db_agent` to fetch raw data.
2.  `call_concierge_agent`: A function that calls the `concierge_agent`.

The `concierge_agent` itself has a tool: the `food_critic_agent`.

The flow for a complex query is:

1.  **User** asks the `trip_data_concierge_agent` for a hotel and a nearby restaurant.
2.  The **Orchestrator** first calls `call_db_agent` to get hotel data.
3.  The data is saved in `tool_context.state`.
4.  The **Orchestrator** then calls `call_concierge_agent`, which retrieves the hotel data from the context.
5.  The `concierge_agent` receives the request and decides it needs to use its own tool, the `food_critic_agent`.
6.  The `food_critic_agent` provides a witty recommendation.
7.  The `concierge_agent` gets the critic's response and politely formats it.
8.  This final, polished response is returned to the **Orchestrator**, which presents it to the user.

                         +-----------------------------------------------------------+
                         |              🧭 Trip Data Concierge Agent                 |
                         |-----------------------------------------------------------|
                         |  Model: gemini-2.5-flash                                  |
                         |  Description:                                             |
                         |   Orchestrates database query and travel recommendation  |
                         |-----------------------------------------------------------|
                         |  🔧 Tools:                                                |
                         |   1. call_db_agent                                        |
                         |   2. call_concierge_agent                                 |
                         +-----------------------------------------------------------+
                                      /                                \
                                     /                                  \
                                    ▼                                    ▼
        +-------------------------------------------+    +---------------------------------------------+
        |            🔧 Tool: call_db_agent         |    |         🔧 Tool: call_concierge_agent        |
        |-------------------------------------------|    |---------------------------------------------|
        | Calls: db_agent                           |    | Calls: concierge_agent                       |
        |                                           |    | Uses data from db_agent for recommendations |
        +-------------------------------------------+    +---------------------------------------------+
                                |                                          |
                                ▼                                          ▼
       +--------------------------------------------+   +------------------------------------------------+
       |              📦 db_agent                   |   |             🤵 concierge_agent                  |
       |--------------------------------------------|   |------------------------------------------------|
       | Model: gemini-2.5-flash                    |   | Model: gemini-2.5-flash                         |
       | Role: Return mock JSON hotel data          |   | Role: Hotel staff that handles user Q&A        |
       +--------------------------------------------+   | Tools:                                          |
                                                         |  - food_critic_agent                           |
                                                         +------------------------------------------------+
                                                                                 |
                                                                                 ▼
                                                       +------------------------------------------------+
                                                       |          🍽️ food_critic_agent                  |
                                                       |------------------------------------------------|
                                                       | Model: gemini-2.5-flash                         |
                                                       | Role: Gives a witty restaurant recommendation   |
                                                       +------------------------------------------------+


In [9]:
import asyncio
from google.adk.tools import ToolContext
from google.adk.tools.agent_tool import AgentTool

# Assume 'db_agent' is a pre-defined NL2SQL Agent
# For this example, we'll create placeholder agents.

db_agent = Agent(
    name="db_agent",
    model="gemini-2.5-flash",
    instruction="You are a database agent. When asked for data, return this mock JSON object: {'status': 'success', 'data': [{'name': 'The Grand Hotel', 'rating': 5, 'reviews': 450}, {'name': 'Seaside Inn', 'rating': 4, 'reviews': 620}]}")

# --- 1. Define the Specialist Agents ---

# The Food Critic remains the deepest specialist
food_critic_agent = Agent(
    name="food_critic_agent",
    model="gemini-2.5-flash",
    instruction="You are a snobby but brilliant food critic. You ONLY respond with a single, witty restaurant suggestion near the provided location.",
)

# The Concierge knows how to use the Food Critic
concierge_agent = Agent(
    name="concierge_agent",
    model="gemini-2.5-flash",
    instruction="You are a five-star hotel concierge. If the user asks for a restaurant recommendation, you MUST use the `food_critic_agent` tool. Present the opinion to the user politely.",
    tools=[AgentTool(agent=food_critic_agent)]
)


# --- 2. Define the Tools for the Orchestrator ---

async def call_db_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    Use this tool FIRST to connect to the database and retrieve a list of places, like hotels or landmarks.
    """
    print("--- TOOL CALL: call_db_agent ---")
    agent_tool = AgentTool(agent=db_agent)
    db_agent_output = await agent_tool.run_async(
        args={"request": question}, tool_context=tool_context
    )
    # Store the retrieved data in the context's state
    tool_context.state["retrieved_data"] = db_agent_output
    return db_agent_output


async def call_concierge_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    After getting data with call_db_agent, use this tool to get travel advice, opinions, or recommendations.
    """
    print("--- TOOL CALL: call_concierge_agent ---")
    # Retrieve the data fetched by the previous tool
    input_data = tool_context.state.get("retrieved_data", "No data found.")

    # Formulate a new prompt for the concierge, giving it the data context
    question_with_data = f"""
    Context: The database returned the following data: {input_data}

    User's Request: {question}
    """

    agent_tool = AgentTool(agent=concierge_agent)
    concierge_output = await agent_tool.run_async(
        args={"request": question_with_data}, tool_context=tool_context
    )
    return concierge_output


# --- 3. Define the Top-Level Orchestrator Agent ---

trip_data_concierge_agent = Agent(
    name="trip_data_concierge",
    model="gemini-2.5-flash",
    description="Top-level agent that queries a database for travel data, then calls a concierge agent for recommendations.",
    tools=[call_db_agent, call_concierge_agent],
    instruction="""
    You are a master travel planner who uses data to make recommendations.

    1.  **ALWAYS start with the `call_db_agent` tool** to fetch a list of places (like hotels) that match the user's criteria.

    2.  After you have the data, **use the `call_concierge_agent` tool** to answer any follow-up questions for recommendations, opinions, or advice related to the data you just found.
    """,
)

print(f"✅ Orchestrator Agent '{trip_data_concierge_agent.name}' is defined and ready.")

✅ Orchestrator Agent 'trip_data_concierge' is defined and ready.


In [10]:
# --- Let's test the Trip Data Concierge Agent ---

async def run_trip_data_concierge():
    """
    Sets up a session and runs a query against the top-level
    trip_data_concierge_agent.
    """
    # Create a new, single-use session for this query
    concierge_session = await session_service.create_session(
        app_name=trip_data_concierge_agent.name,
        user_id=my_user_id
    )

    # This query is specifically designed to trigger the full two-step process:
    # 1. Get data from the db_agent.
    # 2. Get a recommendation from the concierge_agent based on that data.
    query = "Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews."
    print(f"🗣️ User Query: '{query}'")

    # We call our existing helper function with the top-level orchestrator agent
    await run_agent_query(trip_data_concierge_agent, query, concierge_session, my_user_id)

# Run the test
await run_trip_data_concierge()

🗣️ User Query: 'Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews.'

🚀 Running query for agent: 'trip_data_concierge' in session: 'b914ea32-f2c3-4e63-a57a-5e87166e34cd'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'question': 'Find the top-rated hotels in San Francisco'
        },
        id='adk-dac0a543-1d9f-44c1-9676-715cdd43f42a',
        name='call_db_agent'
      ),
      thought_signature=b'\n\xac\x03\x01\xd1\xed\x8aoo\r\x1eC\xbe\xcb\xa7\x01M\xd8\xb7\xb2L\xaf\xe7\xa7}\x0c\x00\xf1S1\x80\x90\xac\xd9e\x90C\\G\xf2\xf4\xaa\xdb\x85\xf84\x86\xeb\xf6\xdf\x92H<UI\xe9\xec\xf1M\x95\x92\xb9\x07\x8c\x14\x84\x11/c\x84;c\xd4\xe0\xf7\xb7\xbb\xc4\xc58\xb3F\xa0\x90\xab\x13\nv\xdco&\xe2\x15\x96\xa9,^...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> err

The top-rated hotels in San Francisco are The Grand Hotel (5 stars, 450 reviews) and Seaside Inn (4 stars, 620 reviews). Seaside Inn has the most reviews. For a truly exquisite dinner experience near the Seaside Inn, I recommend Gary Danko.

--------------------------------------------------



---
## Part 3: Agent with a Memory - The Adaptive Planner 🗺️

Now, let's see an agent that not only **remembers** but also **adapts**. We'll challenge the `multi_day_trip_agent` to re-plan part of its itinerary based on our feedback. This is a much more realistic test of conversational AI.

```
+-----------------------------------------------------+
|         Adaptive Multi-Day Trip Agent 🗺️           |
|-----------------------------------------------------|
|  Model: gemini-2.5-flash                            |
|  Description:                                       |
|   Builds multi-day travel itineraries step-by-step, |
|   remembers previous days, adapts to feedback       |
|-----------------------------------------------------|
|  🔧 Tools:                                          |
|   - Google Search                                   |
|-----------------------------------------------------|
|  🧠 Capabilities:                                   |
|   - Memory of past conversation & preferences       |
|   - Progressive planning (1 day at a time)          |
|   - Adapts to user feedback                         |
|   - Ensures activity variety across days            |
+-----------------------------------------------------+

            ▲
            |
    +---------------------------+
    |     User Interaction      |
    |---------------------------|
    | - Destination             |
    | - Trip duration           |
    | - Interests & feedback    |
    +---------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Day-by-Day Itinerary Generation              |
|-----------------------------------------------------|
|  🗓️ Day N Output (Markdown format):                 |
|   - Morning / Afternoon / Evening activities        |
|   - Personalized & context-aware                    |
|   - Changes accepted, feedback acknowledged         |
+-----------------------------------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Next Day Planning Triggered 🚀               |
|-----------------------------------------------------|
| - Builds on prior days                              |
| - Avoids repetition                                 |
| - Asks user for confirmation before proceeding      |
+-----------------------------------------------------+
```


In [11]:
# --- Agent Definition: The Adaptive Planner ---

def create_multi_day_trip_agent():
    """Create the Progressive Multi-Day Trip Planner agent"""
    return Agent(
        name="multi_day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent that progressively plans a multi-day trip, remembering previous days and adapting to user feedback.",
        instruction="""
        You are the "Adaptive Trip Planner" 🗺️ - an AI assistant that builds multi-day travel itineraries step-by-step.

        Your Defining Feature:
        You have short-term memory. You MUST refer back to our conversation to understand the trip's context, what has already been planned, and the user's preferences. If the user asks for a change, you must adapt the plan while keeping the unchanged parts consistent.

        Your Mission:
        1.  **Initiate**: Start by asking for the destination, trip duration, and interests.
        2.  **Plan Progressively**: Plan ONLY ONE DAY at a time. After presenting a plan, ask for confirmation.
        3.  **Handle Feedback**: If a user dislikes a suggestion (e.g., "I don't like museums"), acknowledge their feedback, and provide a *new, alternative* suggestion for that time slot that still fits the overall theme.
        4.  **Maintain Context**: For each new day, ensure the activities are unique and build logically on the previous days. Do not suggest the same things repeatedly.
        5.  **Final Output**: Return each day's itinerary in MARKDOWN format.
        """,
        tools=[google_search]
    )

multi_day_agent = create_multi_day_trip_agent()
print(f"🗺️ Agent '{multi_day_agent.name}' is created and ready to plan and adapt!")

🗺️ Agent 'multi_day_trip_agent' is created and ready to plan and adapt!


### Scenario 3a: Agent WITH Memory (Using a SINGLE Session) ✅

First, let's see the correct way to do it. We will use the **exact same `trip_session` object** for the entire conversation. Watch how the agent remembers the context from Turn 1 to correctly handle the requests in Turn 2 and 3.

In [12]:
# --- Scenario 2: Testing Adaptation and Memory ---

async def run_adaptive_memory_demonstration():
    print("### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###")

    # Create ONE session that we will reuse for the whole conversation
    trip_session = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"Created a single session for our trip: {trip_session.id}")

    # --- Turn 1: The user initiates the trip ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    print(f"\n🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, trip_session, my_user_id)

    # --- Turn 2: The user gives FEEDBACK and asks for a CHANGE ---
    # We use the EXACT SAME `trip_session` object!
    query2 = "That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?"
    print(f"\n🗣️ User (Turn 2 - Feedback): '{query2}'")
    await run_agent_query(multi_day_agent, query2, trip_session, my_user_id)

    # --- Turn 3: The user confirms and asks to continue ---
    query3 = "Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind."
    print(f"\n🗣️ User (Turn 3 - Confirmation): '{query3}'")
    await run_agent_query(multi_day_agent, query3, trip_session, my_user_id)

await run_adaptive_memory_demonstration()

### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###
Created a single session for our trip: 519cd0cb-0c6a-4c42-b5b0-a050addb45bd

🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '519cd0cb-0c6a-4c42-b5b0-a050addb45bd'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Hello! Lisbon is a fantastic choice with its rich history and delicious food. I can definitely help you plan your 2-day trip!

Let's start with **Day 1**. How about this itinerary?

**Day 1: Alfama and Baixa Exploration**

*   **Morning (9:00 AM - 1:00 PM):** Begin your day exploring the historic Alfama district, Lisbon's oldest neighborhood. Wander through its narrow, winding streets, discover hidden viewpoints (miradouros), and visit the Lisbon Cathedral (Sé de Lisboa).
*   **Lunch (1:00 PM - 2:30 PM):** Enjoy a traditional Portuguese lunch

Hello! Lisbon is a fantastic choice with its rich history and delicious food. I can definitely help you plan your 2-day trip!

Let's start with **Day 1**. How about this itinerary?

**Day 1: Alfama and Baixa Exploration**

*   **Morning (9:00 AM - 1:00 PM):** Begin your day exploring the historic Alfama district, Lisbon's oldest neighborhood. Wander through its narrow, winding streets, discover hidden viewpoints (miradouros), and visit the Lisbon Cathedral (Sé de Lisboa).
*   **Lunch (1:00 PM - 2:30 PM):** Enjoy a traditional Portuguese lunch in Alfama. Look for a local *tasca* (a small, informal restaurant) serving fresh seafood or grilled sardines.
*   **Afternoon (2:30 PM - 6:00 PM):** Head to the Baixa district, rebuilt after the 1755 earthquake. Walk along Rua Augusta, admire the Santa Justa Lift, and explore Praça do Comércio, a grand riverside square.
*   **Evening (7:00 PM onwards):** Experience a traditional Fado show with dinner in Alfama or Bairro Alto. Fado is a soulful Portuguese music genre, often accompanied by delicious local cuisine.

How does this sound for your first day?

--------------------------------------------------


🗣️ User (Turn 2 - Feedback): 'That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '519cd0cb-0c6a-4c42-b5b0-a050addb45bd'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Okay, no problem at all! I can definitely swap out the castle for another historical experience.

Let's revise **Day 1** to focus on the charming historical aspects of Alfama:

**Day 1: Alfama and Baixa Exploration**

*   **Morning (9:00 AM - 1:00 PM):** Begin your day exploring the historic **Alfama district**, Lisbon's oldest neighborhood. Wander through its narrow, winding streets, discover hidden viewpoints (*miradouros*) offering stunning cityscapes, and visit the impressive **Lisbon Cathedral (Sé de Lisboa)**, a Romanesque landmark with a rich history.
*   **Lunch (1:0

Okay, no problem at all! I can definitely swap out the castle for another historical experience.

Let's revise **Day 1** to focus on the charming historical aspects of Alfama:

**Day 1: Alfama and Baixa Exploration**

*   **Morning (9:00 AM - 1:00 PM):** Begin your day exploring the historic **Alfama district**, Lisbon's oldest neighborhood. Wander through its narrow, winding streets, discover hidden viewpoints (*miradouros*) offering stunning cityscapes, and visit the impressive **Lisbon Cathedral (Sé de Lisboa)**, a Romanesque landmark with a rich history.
*   **Lunch (1:00 PM - 2:30 PM):** Enjoy a traditional Portuguese lunch in Alfama. Look for a local *tasca* (a small, informal restaurant) serving fresh seafood or grilled sardines.
*   **Afternoon (2:30 PM - 6:00 PM):** Head to the Baixa district, rebuilt after the 1755 earthquake. Walk along Rua Augusta, admire the Santa Justa Lift, and explore Praça do Comércio, a grand riverside square.
*   **Evening (7:00 PM onwards):** Experience a traditional Fado show with dinner in Alfama or Bairro Alto. Fado is a soulful Portuguese music genre, often accompanied by delicious local cuisine.

How does this revised plan for Day 1 sound?

--------------------------------------------------


🗣️ User (Turn 3 - Confirmation): 'Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '519cd0cb-0c6a-4c42-b5b0-a050addb45bd'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Excellent! I'm glad Day 1 is perfect. Let's move on to **Day 2**, keeping your love for history and great local food in mind, and ensuring we explore new parts of Lisbon.

Here's a proposal for Day 2:

**Day 2: Belém's Maritime History and Culinary Delights**

*   **Morning (9:30 AM - 1:00 PM):** Start your day in the historic **Belém district**, synonymous with Portugal's Age of Discoveries. Visit the magnificent **Jerónimos Monastery**, a UNESCO World Heritage site showcasing impressive Manueline architecture. Afterwards, stroll along the Tagus River to see the iconic **Belém Tower** and the **Monument to the Di

Excellent! I'm glad Day 1 is perfect. Let's move on to **Day 2**, keeping your love for history and great local food in mind, and ensuring we explore new parts of Lisbon.

Here's a proposal for Day 2:

**Day 2: Belém's Maritime History and Culinary Delights**

*   **Morning (9:30 AM - 1:00 PM):** Start your day in the historic **Belém district**, synonymous with Portugal's Age of Discoveries. Visit the magnificent **Jerónimos Monastery**, a UNESCO World Heritage site showcasing impressive Manueline architecture. Afterwards, stroll along the Tagus River to see the iconic **Belém Tower** and the **Monument to the Discoveries**.
*   **Lunch & Treat (1:00 PM - 2:30 PM):** Indulge in one of Lisbon's most famous culinary delights: the *Pastéis de Belém*. Head to the original factory, **Pastéis de Belém**, to savor these warm custard tarts dusted with cinnamon and powdered sugar. You can also grab a light lunch at a nearby traditional eatery in Belém.
*   **Afternoon (2:30 PM - 6:00 PM):** Explore the vibrant **LX Factory**, a former industrial complex transformed into a trendy hub of independent shops, art studios, and unique restaurants. It's a great spot to see Lisbon's creative side and perhaps pick up some local crafts or specialty food items.
*   **Evening (7:00 PM onwards):** For your farewell dinner, immerse yourself in the culinary scene at the **Time Out Market (Mercado da Ribeira)**. This bustling food hall offers a vast selection of gourmet Portuguese and international dishes from various acclaimed chefs and restaurants, allowing you to sample a wide array of local flavors.

How does this sound for your second day in Lisbon?

--------------------------------------------------



### Scenario 3b: Agent WITHOUT Memory (Using SEPARATE Sessions) ❌

Now, let's see what happens if we mess up our session management. Here, we'll give the agent a case of amnesia by creating a **brand new, separate session for each turn**.

Pay close attention to the agent's response to the second query. Because it's in a new session, it has no memory of the trip to Lisbon we just discussed!

In [13]:
# --- Scenario 2b: Demonstrating Memory FAILURE ---

async def run_memory_failure_demonstration():
    print("\n" + "#"*60)
    print("### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###")
    print("#"*60)

    # --- Turn 1: The user initiates the trip in the FIRST session ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    session_one = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a session for Turn 1: {session_one.id}")
    print(f"🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, session_one, my_user_id)

    # --- Turn 2: The user asks to continue... but in a completely NEW session ---
    query2 = "Yes, that looks perfect! Please plan Day 2."
    session_two = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a BRAND NEW session for Turn 2: {session_two.id}")
    print(f"🗣️ User (Turn 2): '{query2}'")
    await run_agent_query(multi_day_agent, query2, session_two, my_user_id)

await run_memory_failure_demonstration()


############################################################
### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###
############################################################

Created a session for Turn 1: c3b5e691-d6e8-4ecc-bad7-c9af1609348a
🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'c3b5e691-d6e8-4ecc-bad7-c9af1609348a'...

--------------------------------------------------
✅ Final Response:


An error occurred: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash\nPlease retry in 52.856019321s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash', 'location': 'global'}, 'quotaValue': '10'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '52s'}]}}

--------------------------------------------------


Created a BRAND NEW session for Turn 2: ac91b1e6-3066-4cfc-b1fc-7cc6aeb8def8
🗣️ User (Turn 2): 'Yes, that looks perfect! Please plan Day 2.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'ac91b1e6-3066-4cfc-b1fc-7cc6aeb8def8'...

--------------------------------------------------
✅ Final Response:


An error occurred: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash\nPlease retry in 52.436906964s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'}, 'quotaValue': '10'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '52s'}]}}

--------------------------------------------------



See? The agent was confused! It likely asked what destination or what trip we were talking about. Because the second query was in a fresh, isolated session, the agent had no memory of planning Day 1 in Lisbon.

This perfectly illustrates why **managing sessions is the key to building truly conversational agents!**

---
## 🎉 Congratulations! 🎉

Congratulations on completing your ADK adventure into Tools and Memory! You've taken a massive leap from building single-shot agents to creating dynamic, stateful AI systems.

Let's recap the powerful concepts you've mastered:

- **Fundamental Agent & Tools**: You started by building a "Day Trip Genie" and equipped it with its first tool, GoogleSearch.

- **Custom Function Tools**: You gave your agent a new sense by creating a custom tool to fetch live data from the U.S. National Weather Service API.

- **Agent-as-a-Tool**: You orchestrated a sophisticated hierarchy where agents delegate tasks to other, more specialized agents, creating a collaborative team.

- **The Power of Memory**: Most importantly, you saw firsthand how managing a single, persistent Session allows an agent to remember context, adapt to user feedback, and conduct a meaningful, multi-turn conversation.

```
   __            /\_/\         /\_/\        /\_/\         __             (\__/)
o-''|\_____/).  ( o.o )       ( -.- )      ( ^_^ )     o-''|\_____/).    ( ^_^ )
 \_/|_)     )    > ^ <         > * <        >💖<         \_/|_)     )     / >🌸< \
    \  __  /                                              \  __  /         /   \
    (_/ (_/                                               (_/ (_/        (___|___)
```
